## Elements of Statistical Learning (ESL) Final Project

**Our Machine Learning Question:**

**How can we improve the accuracy of predicting** which diabetic patients are likely to be readmitted to the hospital within 30 days across different classifiers **by addressing the severe class imbalance in the dataset?**
The dataset we chose is on “Early Readmission Prediction of Patients Diagnosed with Diabetes,” which has a significant class imbalance between early admitted patients and others. The positive class for our classification (early admitted patients) is underrepresented with a ratio of 1-to-9.

The techniques we are considering implementing to overcome the class imbalance in the dataset are as follows:
1. SMOTE (Synthetic Minority Oversampling Technique)
2. Class weights
3. Ensemble methods

All the above techniques will be compared across different classifiers to answer the following question: How well do different models (e.g., Logistic Regression, Random Forest, SVM) handle the imbalance and predict early readmissions?

Recommendations from Professor David:

- Check how different models react to class imbalance prior to implementing balancing techniques.
- Also, check if the balancing techniques actually cause that the same objects are misclassified, or that suddenly also other objects go wrong (that used to be classified well).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Data Loading and Exploration

The chosen dataset represents ten years (1999-2008) of clinical care at 130 US hospitals and integrated delivery networks. Each row concerns hospital records of patients diagnosed with diabetes, who underwent laboratory, medications, and stayed up to 14 days.

Link to the dataset: https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008

**Our Aim:** Identifying patients at risk of being readmitted within 30 days of discharge.

In [2]:
dataset = pd.read_csv('diabetic_data.csv')
dataset.head(20)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,...,No,Steady,No,No,No,No,No,No,Yes,>30
6,55842,84259809,Caucasian,Male,[60-70),?,3,1,2,4,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,63768,114882984,Caucasian,Male,[70-80),?,1,1,7,5,...,No,No,No,No,No,No,No,No,Yes,>30
8,12522,48330783,Caucasian,Female,[80-90),?,2,1,4,13,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,15738,63555939,Caucasian,Female,[90-100),?,3,3,4,12,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [3]:
print(dataset.columns)

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')


In [4]:
target_distribution = dataset['readmitted'].value_counts()

print("Distribution of 'readmitted' target column:")
print(target_distribution)

target_percentage = dataset['readmitted'].value_counts(normalize=True) * 100
print("\nPercentage distribution of 'readmitted' target column:")
print(f"{round(target_percentage, 2)}")

Distribution of 'readmitted' target column:
readmitted
NO     54864
>30    35545
<30    11357
Name: count, dtype: int64

Percentage distribution of 'readmitted' target column:
readmitted
NO     53.91
>30    34.93
<30    11.16
Name: proportion, dtype: float64


### Binary Classification Formulation

There are three classes in the target 'readmitted' column:
- Class 1: <30 (early admission)
- Class 2: NO (no admission)
- Class 3: >30 (late admission)

For the purpose of our early admission prediction problem, we will use binary classification because we are looking for a way to classify early admissions correctly in this problem. Therefore, for our problem, the "No" and ">30" cases mean the same and can be combined into the same group.

Hence, the **binary classification** formulation can be described as follows:
- Class 1: <30 (early admission)
- Class 2: NO and >30 (no admission and late admission)

### Understanding the Imbalance in the Dataset

After formulating the dataset as a binary classification problem, it can be seen that the combined percentage of the "No" and ">30" classes is approximately 88.84%, whereas the percentage distribution of the early admission class "<30" is 11.16%.

Hence, it can be seen that there is a severe imbalance in the dataset. The positive class for our classification (early admitted patients) is underrepresented, with a ratio of 1-to-9.

## TODO During the Christmas Break

Link to the Google sheets for writing down comments on the dataset columns and rationalizing the implemented approaches:
https://docs.google.com/spreadsheets/d/1wQvVQijqFmdOWjL4nJtSQ5hRea5tVSDQo9-br04bgew/edit?usp=sharing

## Data Exploration Ideas

### 1. Missing Value Handling
- **Indicator of Missing Values**: Missing values are represented by `?` in the dataset.
- **Imputation Strategies**:
  - **Baseline Approach: Deleting the Rows with Missing Values:** Remove rows that contain at least one missing value in any column.
  - **Correlation-based Imputation**: Identify which features are highly correlated and impute missing values accordingly.
  - **Statistical Methods**:
    - For **categorical features**: Use the most frequent class (mode).
    - For **numerical features**: Use the mean, median, or a neighbor-based approach (e.g., K-Nearest Neighbors).
      - **Note**: Research the name of the neighbor-based imputation technique (e.g., KNN Imputation).
- **Documentation**: Record the rationale behind the chosen imputation strategy.

### 2. Feature Reduction & Extraction
- **Initial Cleanup**:
  - Drop non-informative columns such as:
    - ID fields.
    - Columns with constant values (e.g., same value for all rows).
- **Rationale**: Clearly document why specific columns were dropped.

- **Dimensionality Reduction**:
  - Implement methods like **Principal Component Analysis (PCA)**.
  - **Library**: Check if Scikit-learn has built-in functions for feature extraction or reduction.
  - Alternatively, develop custom logic for feature reduction based on feature correlation.
- **Rationale**: Justify the use of any dimensionality reduction method and its impact on the dataset.

---

## Data Preprocessing Ideas

### 1. Normalization
- **Research Questions**:
  - Which features require normalization?
    - Should normalization apply only to the target class or to all features?
- **Initial Assumption**:
  - Normalization should be applied to all features since some classifiers are sensitive to feature scaling.
- **Documentation**: Record the decision and reasoning behind normalization.

### 2. Handling Class Imbalance
- **Techniques to Address Imbalance**:
  1. **SMOTE (Synthetic Minority Oversampling Technique)**: Generate synthetic samples for the minority class.
  2. **Class Weights**: Assign higher weights to the minority class during training.
  3. **Ensemble Methods**: Use techniques like Random Forest or Bagging that are robust to imbalanced data.
- **Classifier Dependency**:
  - Some imbalance techniques are suitable only for specific classification algorithms.
  - **Documentation**: Clearly note the classifiers compatible with each imbalance-handling technique.

---

## Notes
- Keep a record of all decisions and approaches in the notebook or Markdown.
- Justify each step with reasoning or research findings for transparency and reproducibility.

---

# Handling Data Imbalance Techniques

## 1. SMOTE (Synthetic Minority Oversampling Technique)
- **Purpose**: Generates synthetic samples for the minority class to balance the dataset.
- **How it Works**: SMOTE creates new samples by interpolating between existing minority class examples.
- **Extensions**:
  - Borderline SMOTE: Focuses on samples near the decision boundary.
  - ADASYN: Generates more synthetic samples for harder-to-learn minority examples.
  - SMOTETomek: Combines SMOTE with Tomek Links to clean the dataset.
- **Resources**:
  - [SMOTE for Imbalanced Classification](https://www.geeksforgeeks.org/smote-for-imbalanced-classification-with-python/): Includes table for when to use each variant.
  - [imblearn.over_sampling.SMOTE Documentation](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html).

---

## 2. Class Weights
- **Purpose**: Adjust model training to account for class imbalance.
- **How it Works**: Assigns higher weights to the minority class, forcing the model to focus on it more.
- **F1 Score Formula**:
  \[ F1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}} \]
  - If F1 = 0, the model performs poorly on the minority class.
- **Implementation**:
  - Use the `class_weight` parameter in classifiers such as Scikit-learn, LightGBM, or CatBoost.
    - Example: For Logistic Regression, set `class_weight='balanced'` or provide manual weights.
- **Resources**:
  - [Improve Class Imbalance Using Class Weights](https://www.analyticsvidhya.com/blog/2020/10/improve-class-imbalance-class-weights/).
  - [How to Set Class Weights in Keras](https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras).

---

## 3. Ensemble Methods
- **Purpose**: Combines multiple classifiers to improve performance and handle class imbalance.
- **Techniques**:
  - **Data-Level Approaches**:
    - **Undersampling**: Reduces the majority class size.
    - **Oversampling**: Increases the minority class size.
    - **Hybrid Approaches**: Combines under and oversampling methods.
  - **Algorithm-Level Techniques**:
    - **Cost-Sensitive Learning**: Assigns different misclassification costs to classes.
    - **Threshold-Moving**: Adjusts the decision threshold to favor the minority class.
  - **Ensemble Learning Methods**:
    - **Bagging**: SMOTEBagging – Combines SMOTE with bagging methods.
    - **Boosting**: RUSBoost – Applies random undersampling with boosting.
    - **Stacking**: EasyEnsemble – Combines multiple models with data resampling.
    - **Hybrid Methods**: Mix bagging + boosting, hybrid sampling + ensemble learning, or dynamic selection + preprocessing.
- **Resources**:
  - [Ensemble Techniques for Class Imbalance](https://thecontentfarm.net/ensemble-techniques-for-handling-class-imbalance/).


### Setting up the Dataset

- Merge the ">30" and "NO" categories of the target variable
- Split the dataset into training and test sets first, ensuring that all preprocessing steps are based on the training data. This will prevent **data leakage**. In other words, information from the test set will not influence the model during training.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Merging ">30" and "NO" (not readmitted) categories of the target variable
dataset['readmitted'] = dataset['readmitted'].apply(lambda x: 1 if x == '<30' else 0)

dataset.head(20)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,0
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,0
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,0
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,0
5,35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,...,No,Steady,No,No,No,No,No,No,Yes,0
6,55842,84259809,Caucasian,Male,[60-70),?,3,1,2,4,...,No,Steady,No,No,No,No,No,Ch,Yes,0
7,63768,114882984,Caucasian,Male,[70-80),?,1,1,7,5,...,No,No,No,No,No,No,No,No,Yes,0
8,12522,48330783,Caucasian,Female,[80-90),?,2,1,4,13,...,No,Steady,No,No,No,No,No,Ch,Yes,0
9,15738,63555939,Caucasian,Female,[90-100),?,3,3,4,12,...,No,Steady,No,No,No,No,No,Ch,Yes,0


In [6]:
# Isolate target column
X = dataset.drop('readmitted', axis=1) # inplace=False (default)
y = dataset['readmitted']

# Splitting the dataset in training and test sets
# From this point on, we will only use the training set until we test the models
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# TODO grouping? to prevent data leakage
# We can't have information of the same person being separated in training and test sets

"""
# Scaling
# Scaling could only be implemented after we convert the categorical variables into numeric.
scaler = StandardScaler()
scaler.fit(X_train)  # Fit on training data only
X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
"""

data = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1) # Training dataset
test_data = pd.concat([X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1) # Test dataset

print("Shape of the original dataset:", dataset.shape)
print("Shape of the training dataset (data):", data.shape)
print("Shape of the test dataset (test_data):", test_data.shape)

Shape of the original dataset: (101766, 50)
Shape of the training dataset (data): (81412, 50)
Shape of the test dataset (test_data): (20354, 50)


### Missing Value Handling

In [7]:
missing_values = data.isnull().sum()
columns_with_missing_values = missing_values[missing_values > 0]
print(f"Columns with missing values:\n{columns_with_missing_values}")

Columns with missing values:
max_glu_serum    77130
A1Cresult        67793
dtype: int64


It is observed from the dataset that in some columns, missing values are represented by the '?' character. Hence, a second data exploration step for analyzing which columns contain the question mark character and their counts is conducted.

In [8]:
question_marks = (data == '?').sum()
columns_with_question_marks = question_marks[question_marks > 0]
print(f"Columns containing question marks and their counts:\n{columns_with_question_marks}")

Columns containing question marks and their counts:
race                  1811
weight               78916
payer_code           32157
medical_specialty    40057
diag_1                  13
diag_2                 281
diag_3                1136
dtype: int64


Based on this observation, we decided to find the summation of missing values (both NaN and ? characters).

In [9]:
total_missing_values = missing_values + question_marks
columns_with_all_missing_values = total_missing_values[total_missing_values > 0]
print("Columns with missing values (including '?' and NaN):")
print(columns_with_all_missing_values)

Columns with missing values (including '?' and NaN):
race                  1811
weight               78916
payer_code           32157
medical_specialty    40057
diag_1                  13
diag_2                 281
diag_3                1136
max_glu_serum        77130
A1Cresult            67793
dtype: int64


In [10]:
# Convert '?'s into pandas NA values
data = data.replace('?', pd.NA)

# Check to see if the missing value summations match with the previous cell's output
missing_values = data.isnull().sum()
columns_with_missing_values = missing_values[missing_values > 0]
print(f"Columns with missing values:\n{columns_with_missing_values}")

assert (total_missing_values == missing_values).all(), "Mismatch in missing value summations!"

# If the assertion is successful, print a success message
print("\nConsistency check passed: Missing value summations match for '?' and NaN handling.")

Columns with missing values:
race                  1811
weight               78916
payer_code           32157
medical_specialty    40057
diag_1                  13
diag_2                 281
diag_3                1136
max_glu_serum        77130
A1Cresult            67793
dtype: int64

Consistency check passed: Missing value summations match for '?' and NaN handling.


#### Baseline Approach: Deleting the Rows with Missing Values

Remove rows that contain at least one missing value in any column.

In [11]:
data_dropped = data.dropna()
print(f"Original dataset shape: {data.shape}")
print(f"Cleaned dataset shape after dropping rows: {data_dropped.shape}")

Original dataset shape: (81412, 50)
Cleaned dataset shape after dropping rows: (0, 50)


It is observed from the output of the previous cell that no rows remain in the dataset when we drop the rows containing a missing value. This suggests that each row contains at least one missing value.

Hence, a more sophisticated missing-value handling is needed.

Firstly, the following three columns contain between approximately 83% - 97% missing values:
- weight               (78916 missing values out of 81412)
- max_glu_serum        (77130 missing values out of 81412)
- A1Cresult            (67793 missing values out of 81412)

Hence, they are not informative and can be dropped before continuing with the rest of the analysis.


In [12]:
print(f"Original training dataset shape: {data.shape}")
columns_to_drop = ['weight', 'max_glu_serum', 'A1Cresult']
data = data.drop(columns=columns_to_drop, axis=1)
print(f"Training dataset shape after dropping columns: {data.shape}")

# Drop the same columns from the test data as well
print(f"Original test dataset shape: {test_data.shape}")
test_data = test_data.drop(columns=columns_to_drop, axis=1)
print(f"Test dataset shape after dropping columns: {test_data.shape}")

Original training dataset shape: (81412, 50)
Training dataset shape after dropping columns: (81412, 47)
Original test dataset shape: (20354, 50)
Test dataset shape after dropping columns: (20354, 47)


NOTE: The two columns that contain approximately 50% missing values can also be dropped. Discuss it with your teammates.

- payer_code           (32157 missing values out of 81412)
- medical_specialty    (40057 missing values out of 81412)

In [13]:
# Now, again remove rows that contain at least one missing value in any column.
print(f"Dataset shape before dropping rows: {data.shape}")
data_dropped = data.dropna()
print(f"Cleaned dataset shape after dropping rows: {data_dropped.shape}")

Dataset shape before dropping rows: (81412, 47)
Cleaned dataset shape after dropping rows: (21409, 47)


**Observation**

Since a significant amount (a forth) of all entries have been deleted, *Deleting the Rows with Missing Values* is not the most reasonable approach but it'll serve as a baseline for the filling of missing values.

#### Correlation-based Imputation

Identify which features are highly correlated and impute missing values accordingly.

In [14]:
print("Column types before applying mapping:")
print(data.dtypes[:25])

Column types before applying mapping:
encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
dtype: object


In [15]:
print("Column types before applying mapping:")
print(data.dtypes[25:])

# First, apply mapping to non-numeric columns,
# prior to computing the correlation between the features

data_encoded = data.copy()
for column in data.select_dtypes(include=['object', 'category']).columns:
    data_encoded[column] = data_encoded[column].astype('category').cat.codes

Column types before applying mapping:
glimepiride                 object
acetohexamide               object
glipizide                   object
glyburide                   object
tolbutamide                 object
pioglitazone                object
rosiglitazone               object
acarbose                    object
miglitol                    object
troglitazone                object
tolazamide                  object
examide                     object
citoglipton                 object
insulin                     object
glyburide-metformin         object
glipizide-metformin         object
glimepiride-pioglitazone    object
metformin-rosiglitazone     object
metformin-pioglitazone      object
change                      object
diabetesMed                 object
readmitted                   int64
dtype: object


In [16]:
print("Column types after applying mapping:")
print(data_encoded.dtypes[:25])

Column types after applying mapping:
encounter_id                int64
patient_nbr                 int64
race                         int8
gender                       int8
age                          int8
admission_type_id           int64
discharge_disposition_id    int64
admission_source_id         int64
time_in_hospital            int64
payer_code                   int8
medical_specialty            int8
num_lab_procedures          int64
num_procedures              int64
num_medications             int64
number_outpatient           int64
number_emergency            int64
number_inpatient            int64
diag_1                      int16
diag_2                      int16
diag_3                      int16
number_diagnoses            int64
metformin                    int8
repaglinide                  int8
nateglinide                  int8
chlorpropamide               int8
dtype: object


In [17]:
print("Column types after applying mapping:")
print(data_encoded.dtypes[25:])

Column types after applying mapping:
glimepiride                  int8
acetohexamide                int8
glipizide                    int8
glyburide                    int8
tolbutamide                  int8
pioglitazone                 int8
rosiglitazone                int8
acarbose                     int8
miglitol                     int8
troglitazone                 int8
tolazamide                   int8
examide                      int8
citoglipton                  int8
insulin                      int8
glyburide-metformin          int8
glipizide-metformin          int8
glimepiride-pioglitazone     int8
metformin-rosiglitazone      int8
metformin-pioglitazone       int8
change                       int8
diabetesMed                  int8
readmitted                  int64
dtype: object


In [18]:
# Check the first 5 rows of the mapped data to see the encodings
print(data_encoded.head)

<bound method NDFrame.head of        encounter_id  patient_nbr  race  gender  age  admission_type_id  \
0         144530250     25163334     0       0    7                  1   
1         170268756     23892489     3       1    5                  1   
2         107759346     18937773     2       1    6                  2   
3         163655958     25327296     0       0    5                  1   
4          69826056      6636825     0       0    3                  1   
...             ...          ...   ...     ...  ...                ...   
81407     182788392     84985335     2       1    8                  1   
81408     280574292    104329206     2       0    8                  1   
81409     400183682     31529736     2       1    7                  1   
81410     176062908     24308937     0       1    4                  3   
81411     240447882    102716397     0       0    7                  1   

       discharge_disposition_id  admission_source_id  time_in_hospital  \
0      

In [19]:
# Now, compute the correlation matrix using the encoded data

correlation_matrix = data_encoded.corr()
print("Correlation Matrix:")
print(correlation_matrix)

Correlation Matrix:
                          encounter_id  patient_nbr      race    gender  \
encounter_id                  1.000000     0.513119  0.081348  0.007470   
patient_nbr                   0.513119     1.000000  0.148955  0.008538   
race                          0.081348     0.148955  1.000000  0.055652   
gender                        0.007470     0.008538  0.055652  1.000000   
age                           0.071424     0.072798  0.116916 -0.049352   
admission_type_id            -0.159571    -0.012429  0.100863  0.015408   
discharge_disposition_id     -0.133095    -0.137038  0.008952 -0.022655   
admission_source_id          -0.114630    -0.034521  0.031070 -0.000991   
time_in_hospital             -0.061111    -0.022195 -0.013439 -0.030128   
payer_code                    0.437275     0.226177  0.035961 -0.000639   
medical_specialty            -0.168099    -0.136822 -0.030110  0.011782   
num_lab_procedures           -0.023834     0.017674 -0.023255 -0.003833   
num_p

In [20]:
def compute_high_correlations(correlation_threshold, corr_matrix): # TODO changed the name from correlation_matrix -M
    """
        Method for calculating highly correlated features above a given threshold
    """
    highly_correlated_features = corr_matrix[(corr_matrix.abs() > correlation_threshold) & (corr_matrix != 1.0)]
    print(f"\nHighly Correlated Features (Threshold > {correlation_threshold}):")
    print(highly_correlated_features.dropna(how="all", axis=0).dropna(how="all", axis=1))

In [21]:
compute_high_correlations(correlation_threshold=0.6, corr_matrix=correlation_matrix)
compute_high_correlations(correlation_threshold=0.5, corr_matrix=correlation_matrix)


Highly Correlated Features (Threshold > 0.6):
Empty DataFrame
Columns: []
Index: []

Highly Correlated Features (Threshold > 0.5):
              encounter_id  patient_nbr    change  diabetesMed
encounter_id           NaN     0.513119       NaN          NaN
patient_nbr       0.513119          NaN       NaN          NaN
change                 NaN          NaN       NaN    -0.506879
diabetesMed            NaN          NaN -0.506879          NaN


**Important Observation**

It has been observed that the highest correlation among the features is 0.5 in magnitude (as the absolute values of the correlation matrix are used for computations). No correlations above this threshold have been observed.

Since 0.5 is not considered a high correlation, implementing a correlation-based imputation technique for handling missing values does not seem plausible.

#### Chosen Imputation Approach: Statistical Methods for Missing Value Handling

- Missing values in **categorical features** will be imputed using the most frequent class (mode).

- Missing values in **numerical features** will be imputed using the **k-nearest neighbor (KNN) imputation approach**.

In [22]:
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

print("Shape of training dataset before imputation:", data.shape)
print("Shape of test dataset before imputation:", test_data.shape)

# All NA values are converted into nan for compatibility with sklearn
data_impute = data.replace({pd.NA: np.nan})
test_data_impute = test_data.replace({pd.NA: np.nan})

categorical_columns = data_impute.select_dtypes(include=['object', 'category']).columns
numerical_columns = data_impute.select_dtypes(include=['number']).columns

# Handling missing values in categorical features using Simple Imputer with "most frequent" strategy
categorical_imputer = SimpleImputer(strategy='most_frequent')
data_impute[categorical_columns] = categorical_imputer.fit_transform(data_impute[categorical_columns])
test_data_impute[categorical_columns] = categorical_imputer.transform(test_data_impute[categorical_columns])

# Handling missing values in numerical features using KNN Imputer with the given k value
k = 5 # TODO test different k values with pipeline
numerical_imputer = KNNImputer(n_neighbors=k)
data_impute[numerical_columns] = numerical_imputer.fit_transform(data_impute[numerical_columns])
test_data_impute[numerical_columns] = numerical_imputer.transform(test_data_impute[numerical_columns])

print("Shape of training dataset after imputation:", data_impute.shape)
print("Shape of test dataset after imputation:", test_data_impute.shape)
print("Total missing values in training dataset after imputation:", data_impute.isnull().sum().sum())
print("Total missing values in test dataset after imputation:", test_data_impute.isnull().sum().sum())

print("First 20 Rows of the Imputed Training Dataset:")
data_impute.head(10)

Shape of training dataset before imputation: (81412, 47)
Shape of test dataset before imputation: (20354, 47)
Shape of training dataset after imputation: (81412, 47)
Shape of test dataset after imputation: (20354, 47)
Total missing values in training dataset after imputation: 0
Total missing values in test dataset after imputation: 0
First 20 Rows of the Imputed Training Dataset:


,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,144530250.0,25163334.0,AfricanAmerican,Female,[70-80),1.0,23.0,5.0,11.0,MC,...,No,Up,No,No,No,No,No,Ch,Yes,0.0
1,170268756.0,23892489.0,Hispanic,Male,[50-60),1.0,1.0,7.0,1.0,MD,...,No,Down,No,No,No,No,No,Ch,Yes,0.0
2,107759346.0,18937773.0,Caucasian,Male,[60-70),2.0,1.0,1.0,5.0,UN,...,No,Steady,No,No,No,No,No,No,Yes,0.0
3,163655958.0,25327296.0,AfricanAmerican,Female,[50-60),1.0,1.0,7.0,1.0,MC,...,No,Steady,No,No,No,No,No,Ch,Yes,0.0
4,69826056.0,6636825.0,AfricanAmerican,Female,[30-40),1.0,3.0,5.0,10.0,MC,...,No,No,No,No,No,No,No,No,Yes,0.0
5,313120976.0,43040187.0,Caucasian,Female,[80-90),1.0,1.0,7.0,5.0,MC,...,No,Up,No,No,No,No,No,Ch,Yes,1.0
6,171691062.0,58281750.0,Caucasian,Male,[40-50),6.0,1.0,17.0,1.0,MC,...,No,Steady,No,No,No,No,No,No,Yes,0.0
7,398561546.0,126515066.0,Caucasian,Male,[50-60),1.0,1.0,7.0,5.0,MC,...,No,Up,No,No,No,No,No,Ch,Yes,0.0
8,95994288.0,23675085.0,Caucasian,Male,[60-70),2.0,1.0,1.0,2.0,MC,...,No,No,No,No,No,No,No,No,No,0.0
9,200574168.0,85874328.0,Caucasian,Male,[20-30),1.0,1.0,7.0,1.0,CP,...,No,Down,No,No,No,No,No,Ch,Yes,0.0


In [23]:
print("First 20 Rows of the Imputed Test Dataset:")
test_data_impute.head(10)

First 20 Rows of the Imputed Test Dataset:


,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,410328092.0,44011827.0,Caucasian,Female,[80-90),1.0,1.0,7.0,4.0,MC,...,No,Up,No,No,No,No,No,Ch,Yes,0.0
1,176106348.0,100162476.0,AfricanAmerican,Male,[60-70),1.0,1.0,7.0,5.0,MC,...,No,Steady,No,No,No,No,No,No,Yes,0.0
2,179566770.0,23326173.0,Caucasian,Female,[70-80),3.0,22.0,1.0,2.0,MC,...,No,Up,No,No,No,No,No,Ch,Yes,0.0
3,52365486.0,3509442.0,Caucasian,Male,[50-60),3.0,18.0,1.0,5.0,?,...,No,No,No,No,No,No,No,No,No,1.0
4,116295702.0,36829953.0,AfricanAmerican,Male,[70-80),6.0,22.0,17.0,12.0,?,...,No,No,No,No,No,No,No,Ch,Yes,1.0
5,109065468.0,30426867.0,Caucasian,Male,[50-60),5.0,2.0,17.0,1.0,SP,...,No,No,No,No,No,No,No,No,No,0.0
6,73767216.0,6896061.0,AfricanAmerican,Male,[70-80),1.0,1.0,7.0,3.0,?,...,No,No,No,No,No,No,No,No,No,0.0
7,103549302.0,5521644.0,Hispanic,Female,[60-70),1.0,1.0,6.0,4.0,UN,...,No,Steady,No,No,No,No,No,Ch,Yes,0.0
8,116060964.0,23233167.0,Caucasian,Male,[50-60),1.0,22.0,7.0,2.0,MC,...,No,Steady,No,No,No,No,No,Ch,Yes,0.0
9,54874638.0,19797408.0,Hispanic,Male,[50-60),2.0,18.0,1.0,8.0,?,...,No,No,No,No,No,No,No,No,No,0.0


### Feature Reduction & Extraction Approaches

#### 1. Feature Reduction

In [24]:
# Apply mapping to non-numeric columns
mapped_data = data_impute.copy()
test_mapped_data = test_data_impute.copy()

for column in data_impute.select_dtypes(include=['object', 'category']).columns:
    mapped_data[column] = mapped_data[column].astype('category').cat.codes

corr_matrix_mapped = mapped_data.corr()
print("Correlation Matrix for Mapped Data:")
print(corr_matrix_mapped)

Correlation Matrix for Mapped Data:
                          encounter_id  patient_nbr      race    gender  \
encounter_id                  1.000000     0.513119  0.095280  0.007470   
patient_nbr                   0.513119     1.000000  0.143939  0.008538   
race                          0.095280     0.143939  1.000000  0.067023   
gender                        0.007470     0.008538  0.067023  1.000000   
age                           0.071424     0.072798  0.131129 -0.049352   
admission_type_id            -0.159571    -0.012429  0.102666  0.015408   
discharge_disposition_id     -0.133095    -0.137038 -0.006628 -0.022655   
admission_source_id          -0.114630    -0.034521  0.018887 -0.000991   
time_in_hospital             -0.061111    -0.022195 -0.018476 -0.030128   
payer_code                   -0.016498    -0.019987  0.029707  0.001454   
medical_specialty            -0.048140    -0.060200 -0.007533  0.018796   
num_lab_procedures           -0.023834     0.017674 -0.021619 -0

#### Variance Threshold Feature Reduction Approach

In [25]:
from sklearn.feature_selection import VarianceThreshold

X_train_mapped = mapped_data.drop(columns=['readmitted'])

selector = VarianceThreshold(threshold=0)

selector.fit(X_train_mapped)
selector.get_support()

cols = [column for column in X_train_mapped.columns
          if column not in X_train_mapped.columns[selector.get_support()]]

print("Columns removed due to having 0 variance:")
print(cols)

mapped_data.drop(cols, axis=1, inplace=True)
test_mapped_data.drop(cols, axis=1, inplace=True)

Columns removed due to having 0 variance:
['acetohexamide', 'troglitazone', 'examide', 'citoglipton']


#### Correlation-based Feature Reduction Approach

In [26]:
compute_high_correlations(correlation_threshold=0.6, corr_matrix=corr_matrix_mapped)
compute_high_correlations(correlation_threshold=0.5, corr_matrix=corr_matrix_mapped)


Highly Correlated Features (Threshold > 0.6):
Empty DataFrame
Columns: []
Index: []

Highly Correlated Features (Threshold > 0.5):
              encounter_id  patient_nbr    change  diabetesMed
encounter_id           NaN     0.513119       NaN          NaN
patient_nbr       0.513119          NaN       NaN          NaN
change                 NaN          NaN       NaN    -0.506879
diabetesMed            NaN          NaN -0.506879          NaN


**Observation:**

The highest correlation among the features is 0.5 in absolute value. Since 0.5 is not considered a high correlation, implementing a correlation-based feature reduction technique on the imputated data does not seem plausible.

Alternative method:

In [27]:
def remove_high_corr_features(data_set, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_mtrx = data_set.corr()
    # pd.DataFrame(data={'c0': [1, 2, 3], 'c1': [4, 5, 6], 'c2': [7, 8, 9]})

    for i in range(1, len(corr_mtrx.columns)):
        for j in range(0, i):
            # print("(", i, ", ", j, "):", corr_mtrx.iloc[i, j])
            # print(corr_mtrx.columns[j])

            if (abs(corr_mtrx.iloc[i, j]) >= threshold) and (corr_mtrx.columns[j] not in col_corr):
                col_name = corr_mtrx.columns[j] # getting the name of column
                col_corr.add(col_name)
                # print(col_corr) # -> columns that are removed due to having above the threshold absolute correlation

                if col_name in data_set.columns:
                    data_set.drop(columns=col_name, inplace=True)
                    # print(data_set)
                    # print(corr_mtrx) # -> dataset's correlation matrix

    print("Deleted", len(col_corr), "columns with threshold", threshold)
    print(col_corr)

In [28]:
X_train_mapped = mapped_data.drop(columns=['readmitted'])

remove_high_corr_features(X_train_mapped, 0.6)

Deleted 0 columns with threshold 0.6
set()


#### 2. Sequencial Feature Selection

#### 2.1 Forward Selection (Wrapper Approach)

In [29]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression

# NOT TESTED

# TODO model?
lreg = LinearRegression()
ffs = sfs(lreg, k_features=20, forward=True, floating=False, verbose=2, scoring='neg_mean_squared_error')
# TODO select how many features? (1, 47?)
# verbose=2 to show results while processing, see where metric stops improving
# TODO add cross validation?
# TODO best scoring metric?

X_fs = mapped_data.drop(columns=['readmitted'])
y_fs = mapped_data['readmitted']

ffs = ffs.fit(X_fs, y_fs)
feat_names = list(ffs.k_feature_names_) # selected features
print(feat_names)
print(ffs.k_score_) # prediction score of the selected features

# print(ffs.subsets_)
# print('Best subset (indices):', ffs.k_feature_idx_)
# print('Best subset (corresponding names):', ffs.k_feature_names_)

# (pd.DataFrame.from_dict(ffs.get_metric_dict()).T)

# resulting dataset
new_data = mapped_data[feat_names]
new_data['readmitted'] = mapped_data['readmitted']
print("before:",mapped_data.shape, "and after:", new_data.shape, "Forward Selection")

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.4s

[2025-01-13 18:21:15] Features: 1/20 -- score: -0.09644325262314989[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.3s

[2025-01-13 18:21:17] Features: 2/20 -- score: -0.09622076406276106[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.5s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.5s

[2025-01-13 18:21:18] Features: 3/20 -- score: -0.09613922746502306
[2025-01-13 18:21:20] Features: 4/20 -- score: -0.09609831235497515
[2025-01-13 18:21:22] Features: 5/20 -- score: -0.09606478218665748
[2025-01-13 18:21:24] Features: 6/20 -- score: -0.09603948831823483
[2025-01-13 18:21:26] Features: 7/20 -- score: -0.09601634213964576
[2025-01-13 18:21:28] Features: 8/20 -- score: -0.09599399766657202
[2025-01-13 18:21:30] Features: 9/20 -- score: -0.09597298826147625
[2025-01-13 18:21:33] Features: 10/20 -- score: -0.09596622498382501
[2025-01-13 18:21:36] Features: 11/20 -- score: -0.09595926444153066
[2025

['encounter_id', 'patient_nbr', 'age', 'admission_type_id', 'discharge_disposition_id', 'time_in_hospital', 'num_procedures', 'num_medications', 'number_emergency', 'number_inpatient', 'diag_1', 'number_diagnoses', 'metformin', 'glimepiride', 'acarbose', 'insulin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'diabetesMed']
-0.09593351445208953
before: (81412, 43) and after: (81412, 21) Forward Selection



[2025-01-13 18:22:05] Features: 20/20 -- score: -0.09593351445208953/var/folders/dl/c09rkg9x0k5788bhcmlygtv00000gn/T/ipykernel_42042/2316822514.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['readmitted'] = mapped_data['readmitted']


#### 3. Feature Extraction

#### 3.1 Principal Component Analysis (PCA)

In [30]:
from sklearn.decomposition import PCA

# NOT TESTED
# scaled_data = preprocessing.scale(data.T)
# StandardScaler().fit_transform(data.T)

X_train_pca = mapped_data.drop(columns=['readmitted'])
y_train_pca = mapped_data['readmitted']

X_test_pca = test_mapped_data.drop(columns=['readmitted'])
y_test_pca = test_mapped_data['readmitted']

# scikit-learn chooses the minimum number of principal components such that 95 percent of the variance is retained
pca = PCA(.95)
pca.fit(X_train_pca)

X_train_pca = pca.transform(X_train_pca)
X_test_pca = pca.transform(X_test_pca)

# graphs?

# logistic regression?

ValueError: could not convert string to float: 'Caucasian'

### One-Hot Encoding

Encode non-numerical features in the dataset.

In [31]:
from sklearn.preprocessing import OneHotEncoder

data_onehot = data_impute.copy() # Training dataset used for all the preprocessing

# this is already done above
# data_onehot['binary'] = data_onehot['readmitted'].apply(lambda x: 1 if x == '<30' else 0)
# data_onehot.drop(['readmitted'], axis=1, inplace=True)

categorical_features = data_onehot.select_dtypes(include=['object', 'category']).columns
numerical_features = data_onehot.select_dtypes(include=['number']).columns

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder.fit(data_onehot[categorical_features])

encoded_train = encoder.transform(data_onehot[categorical_features])
encoded_train_df = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out(categorical_features))

encoded_test = encoder.transform(test_data_impute[categorical_features])
encoded_test_df = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out(categorical_features))

data_onehot = pd.concat([data_onehot.reset_index(drop=True).drop(columns=categorical_features), encoded_train_df], axis=1)
test_onehot = pd.concat([test_data_impute.reset_index(drop=True).drop(columns=categorical_features), encoded_test_df], axis=1)

print(f"Training Dataset Shape Before Encoding: {data_impute.shape}")
print(f"Training Dataset Shape After Encoding: {data_onehot.shape}")
print(data_onehot.head())

print(f"Test Dataset Shape Before Encoding: {test_data_impute.shape}")
print(f"Test Dataset Shape After Encoding: {test_onehot.shape}")
print(test_onehot.head())

Training Dataset Shape Before Encoding: (81412, 47)
Training Dataset Shape After Encoding: (81412, 2358)
   encounter_id  patient_nbr  admission_type_id  discharge_disposition_id  \
0   144530250.0   25163334.0                1.0                      23.0   
1   170268756.0   23892489.0                1.0                       1.0   
2   107759346.0   18937773.0                2.0                       1.0   
3   163655958.0   25327296.0                1.0                       1.0   
4    69826056.0    6636825.0                1.0                       3.0   

   admission_source_id  time_in_hospital  num_lab_procedures  num_procedures  \
0                  5.0              11.0                45.0             1.0   
1                  7.0               1.0                32.0             4.0   
2                  1.0               5.0                46.0             2.0   
3                  7.0               1.0                36.0             0.0   
4                  5.0          

Onehot encoding significantly increases number of features, may need feature reduction here.

#### Feature Reduction Implemented on One-Hot Encoded Data

First, implement variance threshold approach.

In [ ]:
from sklearn.feature_selection import VarianceThreshold

X_train_onehot = data_onehot.drop(columns=['readmitted'])

selector = VarianceThreshold(threshold=0)

selector.fit(X_train_onehot)
selector.get_support()

cols = [column for column in X_train_onehot.columns
          if column not in X_train_onehot.columns[selector.get_support()]]

print("Columns removed due to having 0 variance:")
print(cols)

data_onehot.drop(cols, axis=1, inplace=True)
test_onehot.drop(cols, axis=1, inplace=True)
print(f"Dataset Shape After variance drop: {data_onehot.shape}")
print(f"Testset Shape After variance drop: {test_onehot.shape}")

Then, implement correlation-based approach.

In [ ]:
def remove_high_corr_features2(data_set, test_set, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_mtrx = data_set.corr()
    # pd.DataFrame(data={'c0': [1, 2, 3], 'c1': [4, 5, 6], 'c2': [7, 8, 9]})

    for i in range(1, len(corr_mtrx.columns)):
        for j in range(0, i):
            # print("(", i, ", ", j, "):", corr_mtrx.iloc[i, j])
            # print(corr_mtrx.columns[j])

            if (abs(corr_mtrx.iloc[i, j]) >= threshold) and (corr_mtrx.columns[j] not in col_corr):
                col_name = corr_mtrx.columns[j] # getting the name of column
                col_corr.add(col_name)
                # print(col_corr) # -> columns that are removed due to having above the threshold absolute correlation

                if col_name in data_set.columns:
                    data_set.drop(columns=col_name, inplace=True)
                    test_set.drop(columns=col_name, inplace=True)
                    # print(data_set)
                    # print(corr_mtrx) # -> dataset's correlation matrix

    print("Deleted columns with threshold", threshold)
    print(col_corr)
    return data_set, test_set

# data_onehot_corr_red = data_onehot.drop(columns=['readmitted'])
# corr_matrix_onehot = data_onehot_corr.corr()
# print("Correlation Matrix for One-Hot Encoded Data:")
# print(corr_matrix)

print("Training Dataset Shape Before Correlation-based Feature Reduction:", data_onehot.shape)
print("Test Set Shape Before Correlation-based Feature Reduction:", test_onehot.shape)
data_onehot, test_onehot = remove_high_corr_features2(data_onehot, test_onehot, 0.9) # TODO what threshold to use?
print("Training Dataset Shape After Correlation-based Feature Reduction:", data_onehot.shape)
print("Test Set Shape After Correlation-based Feature Reduction:", test_onehot.shape)

#### Forward Selection Implemented on One-Hot Encoded Dataset

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression

# NOT TESTED

# TODO model?
lreg = LinearRegression()
sfs1 = sfs(lreg, k_features=4, forward=True, scoring='neg_mean_squared_error')
# TODO select how many features?
# "forward" and "floating" fields
# TODO best scoring metric?

# TODO change data_onehot for the one resulting from feature reduction above
X_fs = data_onehot.drop(columns=['readmitted'])
y_fs = data_onehot['readmitted']

sfs1 = sfs1.fit(X_fs, y_fs)
feat_names = list(sfs1.k_feature_names_) # selected features
print(feat_names)

# print(sfs1.subsets_)
# print(sfs1.k_score_) # prediction score of the selected features
# print('Best subset (indices):', sfs1.k_feature_idx_)
# print('Best subset (corresponding names):', sfs1.k_feature_names_)

# resulting dataset
new_data = data_onehot[feat_names]
new_data['readmitted'] = data_onehot['readmitted']
print("before:",data_onehot.shape, "and after:", new_data.shape, "Forward Selection")

#### Feature Extraction Implemented on One-Hot Encoded Dataset

#### PCA

In [36]:
from sklearn.decomposition import PCA

# NOT TESTED

X_train_pca = data_onehot.drop(columns=['readmitted'])
y_train_pca = data_onehot['readmitted']

X_test_pca = test_onehot.drop(columns=['readmitted'])
y_test_pca = test_onehot['readmitted']

# scikit-learn chooses the minimum number of principal components such that 95 percent of the variance is retained
pca = PCA(.95)
pca.fit(X_train_pca)

X_train_pca = pca.transform(X_train_pca)
X_test_pca = pca.transform(X_test_pca)
# print("Dataset X Shape After PCA:", X_train_pca.shape)
# print("Testset X Shape After PCA:", X_train_pca.shape)

### Imbalance Handling 

#### SMOTE

In [32]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Changed it to reflect the splitting in the beginning
# Used data_onehot and test_onehot for the training and test datasets respectively

X_train = data_onehot.drop(columns=['readmitted'])
y_train = data_onehot['readmitted']

X_test = test_onehot.drop(columns=['readmitted'])
y_test = test_onehot['readmitted']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

print('Original Class Distribution:', Counter(y_train))
print('Over-sampled Class Distribution:', Counter(y_resampled))

# Important Note: The test dataset should reflect the actual class distribution in the population, although it is imbalanced.
# Hence, we don't implement SMOTE technique on the test dataset.

KeyboardInterrupt: 

Classes are now balanced using the SMOTE technique.

#### Undersampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

under_sampler = RandomUnderSampler(random_state=42)

X_under, y_under = under_sampler.fit_resample(X_train, y_train)

print('Original Class Distribution:', Counter(y_train))
print('Under-sampled Class Distribution:', Counter(y_under))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

model = LogisticRegression(random_state=42, max_iter=500)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('\nPerformance on Original Data:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix (Original Data):\n', confusion_matrix(y_test, y_pred))

# Implement balanced class weights
# with the parameter class_weight = "balanced"
model_original = LogisticRegression(random_state=42, class_weight='balanced', max_iter=500)
model_original.fit(X_train, y_train)
y_pred_original = model_original.predict(X_test)

print('\nPerformance on Original Data with Balanced Class Weights:')
print(classification_report(y_test, y_pred_original))
print('Confusion Matrix (Original Data) with Balanced Class Weights:\n', confusion_matrix(y_test, y_pred_original))

model_smote = LogisticRegression(random_state=42, max_iter=500)
model_smote.fit(X_resampled, y_resampled)
y_pred_smote = model_smote.predict(X_test)

print('\nPerformance on SMOTE-Resampled Data:')
print(classification_report(y_test, y_pred_smote))
print('Confusion Matrix (SMOTE Data):\n', confusion_matrix(y_test, y_pred_smote))

model_under = LogisticRegression(random_state=42, max_iter=500)
model_under.fit(X_under, y_under)
y_pred_under = model_under.predict(X_test)

print('\nPerformance on Under-sampled Data:')
print(classification_report(y_test, y_pred_under))
print('Confusion Matrix (Under-sampled Data):\n', confusion_matrix(y_test, y_pred_under))

The most brute force oversampling, it somewhat increases performance on the minority class (not much though), but it also destroys predictions on the majority class, need more experiments with other methods.

Undersampling and class weights all yield similar results.